# Merge de LoRA y Subida a S3

Este notebook mergea un modelo LoRA con su modelo base y sube el resultado a S3.

## Configuración

1. Asegúrate de tener instaladas las dependencias:
   ```bash
   pip install torch transformers peft huggingface-hub boto3
   ```

2. Configura el token de Hugging Face en `KEYS.py` (en la raíz del proyecto)

3. Configura tus credenciales AWS (para subir a S3)


In [1]:
import json
import os
import sys
import shutil
import tempfile
from pathlib import Path

# Agregar el directorio raíz al path para importar KEYS
sys.path.insert(0, str(Path().absolute().parent))

try:
    from KEYS import HF_TOKEN, MODEL_S3_BUCKET
except ImportError:
    print("ERROR: No se encontró KEYS.py. Crea el archivo con HF_TOKEN y MODEL_S3_BUCKET")
    HF_TOKEN = None
    MODEL_S3_BUCKET = "modelo-generador-maia-g8"

try:
    import boto3
    S3_AVAILABLE = True
except ImportError:
    S3_AVAILABLE = False
    print("WARNING: boto3 no está instalado. No se podrá subir a S3.")
    print("Instala con: pip install boto3")

try:
    import torch
    print(f"PyTorch version: {torch.__version__}")
except ImportError as e:
    print(f"ERROR: torch no está instalado. Instala con: pip install torch")
    raise

try:
    from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
    import transformers
    print(f"Transformers version: {transformers.__version__}")
except ImportError as e:
    print(f"ERROR: transformers no está instalado.")
    raise

try:
    from peft import PeftModel
    import peft
    print(f"PEFT version: {peft.__version__}")
except ImportError as e:
    print(f"ERROR: peft no está instalado.")
    raise

try:
    from huggingface_hub import login
except ImportError as e:
    print(f"ERROR: huggingface_hub no está instalado.")
    raise

print("Todas las dependencias están disponibles.\n")


PyTorch version: 2.6.0+cpu


d:\OneDrive\Documents\MAIA\Semestre 4\Despliegue de Soluciones\__REPOS__\Generacion-de-resumenes.-medicos\.venv_generacion\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Transformers version: 4.45.0
PEFT version: 0.10.0
Todas las dependencias están disponibles.



## Configuración de Parámetros

Ajusta estos valores según tu caso:



In [2]:
# Ruta al directorio final del entrenamiento LoRA
LORA_PATH = "generacion/ollama/outputs/meta-llama__Llama-3.2-3B-Instruct-6_epocas/final"

# Nombre del modelo (se detecta automáticamente del LORA_PATH si no se especifica)
MODEL_NAME = None  # Si es None, se usa el nombre del directorio padre de LORA_PATH

# Región de AWS
AWS_REGION = "us-east-1"

print(f"LORA_PATH: {LORA_PATH}")
print(f"MODEL_S3_BUCKET: {MODEL_S3_BUCKET}")
print(f"AWS_REGION: {AWS_REGION}")

# Detectar MODEL_NAME si no se especificó
if MODEL_NAME is None:
    MODEL_NAME = Path(LORA_PATH).parent.name
    print(f"MODEL_NAME detectado automáticamente: {MODEL_NAME}")
else:
    print(f"MODEL_NAME: {MODEL_NAME}")


LORA_PATH: generacion/ollama/outputs/meta-llama__Llama-3.2-3B-Instruct-6_epocas/final
MODEL_S3_BUCKET: modelo-generador-maia-g8
AWS_REGION: us-east-1
MODEL_NAME detectado automáticamente: meta-llama__Llama-3.2-3B-Instruct-6_epocas


## Función de Merge


In [3]:
def merge_lora_with_base(lora_dir: str, output_dir: str):
    """Mergea el LoRA con el modelo base"""
    print(f"Mergeando LoRA desde {lora_dir}...")
    
    # Autenticarse con Hugging Face si hay token disponible
    if HF_TOKEN:
        print("Autenticándose con Hugging Face...")
        try:
            login(token=HF_TOKEN, add_to_git_credential=False)
            print("Autenticación exitosa con Hugging Face")
        except Exception as e:
            print(f"WARNING: No se pudo autenticar con Hugging Face: {e}")
    else:
        print("WARNING: No se encontró HF_TOKEN en KEYS.py")
    
    # Resolver el path (puede ser relativo o absoluto)
    lora_path = Path(lora_dir)
    if not lora_path.is_absolute():
        # Si es relativo, intentar desde el directorio raíz del proyecto
        project_root = Path().absolute().parent
        lora_path = project_root / lora_dir
        print(f"Path relativo detectado, resolviendo desde raíz del proyecto: {lora_path}")
    
    # Verificar que el directorio existe
    if not lora_path.exists():
        print(f"ERROR: El directorio no existe: {lora_path}")
        print(f"Directorio actual de trabajo: {Path.cwd()}")
        print(f"Intentando path absoluto: {Path(lora_dir).absolute()}")
        raise FileNotFoundError(f"No se encontró el directorio: {lora_path}")
    
    cfg_path = lora_path / "adapter_config.json"
    
    if not cfg_path.exists():
        print(f"ERROR: No se encontró adapter_config.json")
        print(f"Directorio buscado: {lora_path}")
        print(f"Archivos en el directorio:")
        for item in lora_path.iterdir():
            print(f"  - {item.name} ({'dir' if item.is_dir() else 'file'})")
        raise FileNotFoundError(f"No se encontró adapter_config.json en {lora_path}")
    
    adapter_cfg = json.loads(cfg_path.read_text(encoding="utf-8"))
    base_model_name = adapter_cfg.get("base_model_name_or_path")
    
    if not base_model_name:
        raise ValueError("adapter_config.json no contiene 'base_model_name_or_path'")
    
    print(f"Modelo base: {base_model_name}")
    print(f"Directorio de salida: {output_dir}")
    
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    print("Cargando tokenizer...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=False, trust_remote_code=True)
    except Exception as e:
        print(f"WARNING: Error al cargar tokenizer desde modelo base: {e}")
        print("Intentando cargar desde directorio LoRA...")
        tokenizer = AutoTokenizer.from_pretrained(str(lora_path), use_fast=False, trust_remote_code=True)
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print("Cargando configuración del modelo base...")
    try:
        config = AutoConfig.from_pretrained(base_model_name, trust_remote_code=True)
    except (ValueError, TypeError, KeyError) as e:
        if "rope_scaling" in str(e):
            print("WARNING: Error de rope_scaling. Ajustando configuración...")
            try:
                import requests
                config_path = Path(tempfile.gettempdir()) / "config_temp.json"
                config_url = f"https://huggingface.co/{base_model_name}/resolve/main/config.json"
                headers = {"Authorization": f"Bearer {HF_TOKEN}"} if HF_TOKEN else {}
                response = requests.get(config_url, headers=headers)
                if response.status_code == 200:
                    config_path.write_text(response.text)
                    config = AutoConfig.from_pretrained(str(config_path.parent), trust_remote_code=True)
                    config_path.unlink()
                else:
                    raise
            except Exception:
                print("WARNING: No se pudo ajustar rope_scaling, continuando...")
                config = AutoConfig.from_pretrained(base_model_name, trust_remote_code=True)
        else:
            raise
    
    print("Cargando modelo base desde HuggingFace...")
    print("NOTA: Esto puede tardar varios minutos (el modelo tiene ~5GB)...")
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        config=config,
        torch_dtype=torch.float32,
        device_map=None,
        low_cpu_mem_usage=False,
        trust_remote_code=True
    )
    
    # Verificar y ajustar tamaño de embeddings si es necesario
    print("Verificando tamaño de vocabulario...")
    try:
        lora_tokenizer_check = AutoTokenizer.from_pretrained(str(lora_path), use_fast=False, trust_remote_code=True)
        lora_vocab_size = len(lora_tokenizer_check) if hasattr(lora_tokenizer_check, '__len__') else lora_tokenizer_check.vocab_size
        base_vocab_size = base_model.config.vocab_size
        
        if lora_vocab_size != base_vocab_size:
            print(f"Ajustando tamaño de embeddings: {base_vocab_size} -> {lora_vocab_size}")
            base_model.resize_token_embeddings(lora_vocab_size)
            print(f"Tamaño de embeddings ajustado exitosamente")
        else:
            print(f"Tamaño de vocabulario coincide: {base_vocab_size}")
    except Exception as e:
        print(f"WARNING: No se pudo verificar tamaño de vocabulario del LoRA: {e}")
        if hasattr(tokenizer, 'vocab_size') and tokenizer.vocab_size != base_model.config.vocab_size:
            print(f"Ajustando tamaño de embeddings: {base_model.config.vocab_size} -> {tokenizer.vocab_size}")
            base_model.resize_token_embeddings(len(tokenizer))
    
    # Limpiar adapter_config.json de campos incompatibles
    print("Verificando y limpiando adapter_config.json...")
    adapter_config_path = lora_path / "adapter_config.json"
    temp_lora_dir = None
    
    if adapter_config_path.exists():
        adapter_config = json.loads(adapter_config_path.read_text(encoding="utf-8"))
        
        # Crear una versión limpia del config con solo los campos que PEFT soporta
        clean_config = {
            "peft_type": adapter_config.get("peft_type", "LORA"),
            "task_type": adapter_config.get("task_type", "CAUSAL_LM"),
            "base_model_name_or_path": adapter_config.get("base_model_name_or_path"),
            "r": adapter_config.get("r"),
            "lora_alpha": adapter_config.get("lora_alpha"),
            "lora_dropout": adapter_config.get("lora_dropout", 0.0),
            "bias": adapter_config.get("bias", "none"),
            "target_modules": adapter_config.get("target_modules"),
            "fan_in_fan_out": adapter_config.get("fan_in_fan_out", False),
            "inference_mode": adapter_config.get("inference_mode", True),
            "init_lora_weights": adapter_config.get("init_lora_weights", True),
        }
        
        # Agregar campos opcionales compatibles
        optional_fields = ["modules_to_save", "revision", "alpha_pattern", "rank_pattern"]
        for field in optional_fields:
            if field in adapter_config and adapter_config[field] is not None:
                clean_config[field] = adapter_config[field]
        
        # Verificar si hay diferencias
        removed_fields = set(adapter_config.keys()) - set(clean_config.keys())
        if removed_fields:
            print(f"WARNING: Removiendo campos incompatibles: {sorted(removed_fields)}")
            # Crear un directorio temporal con el config limpio
            temp_lora_dir = Path(tempfile.mkdtemp())
            # Copiar todos los archivos excepto adapter_config.json
            for file in lora_path.iterdir():
                if file.name != "adapter_config.json":
                    if file.is_file():
                        shutil.copy2(file, temp_lora_dir / file.name)
                    else:
                        shutil.copytree(file, temp_lora_dir / file.name)
            # Guardar el config limpio
            with open(temp_lora_dir / "adapter_config.json", 'w', encoding='utf-8') as f:
                json.dump(clean_config, f, indent=2)
            
            lora_path_to_use = temp_lora_dir
            print(f"Usando configuración temporal limpia")
        else:
            lora_path_to_use = lora_path
    else:
        lora_path_to_use = lora_path
    
    print("Cargando adaptador LoRA...")
    try:
        model = PeftModel.from_pretrained(base_model, str(lora_path_to_use), device_map=None)
    finally:
        # Limpiar directorio temporal si se creó
        if temp_lora_dir and temp_lora_dir.exists():
            shutil.rmtree(temp_lora_dir, ignore_errors=True)
    
    print("Mergeando LoRA con modelo base...")
    print("NOTA: Esto puede tardar varios minutos...")
    model = model.merge_and_unload()
    
    print(f"Guardando modelo mergeado en {output_dir}...")
    print("NOTA: Esto puede tardar varios minutos...")
    model.save_pretrained(str(output_path), safe_serialization=True)
    tokenizer.save_pretrained(str(output_path))
    
    print(f"Modelo mergeado guardado exitosamente en {output_dir}")
    return output_path


In [4]:
def upload_to_s3(local_path: Path, s3_bucket: str, s3_key_prefix: str, region: str = "us-east-1"):
    """Sube el modelo mergeado a S3"""
    if not S3_AVAILABLE:
        print("ERROR: boto3 no está disponible. No se puede subir a S3.")
        return False
    
    print(f"\nSubiendo modelo a S3...")
    print(f"  Bucket: {s3_bucket}")
    print(f"  Key prefix: {s3_key_prefix}")
    
    s3_client = boto3.client('s3', region_name=region)
    
    try:
        total_files = sum(1 for _ in local_path.rglob('*') if _.is_file())
        uploaded = 0
        
        for file_path in local_path.rglob('*'):
            if file_path.is_file():
                relative_path = file_path.relative_to(local_path)
                s3_key = f"{s3_key_prefix}/{relative_path}".replace("\\", "/")
                
                print(f"  Subiendo {relative_path}... ({uploaded + 1}/{total_files})")
                s3_client.upload_file(str(file_path), s3_bucket, s3_key)
                uploaded += 1
        
        print(f"\nModelo subido exitosamente a s3://{s3_bucket}/{s3_key_prefix}/")
        return True
    except Exception as e:
        print(f"ERROR al subir a S3: {e}")
        return False


## Ejecutar Merge y Subida


In [5]:
# Crear directorio temporal para el modelo mergeado
# Usar un directorio en el proyecto en lugar del temp de Windows
temp_output = Path("generacion/ollama/temp_merged_model")
temp_output.mkdir(parents=True, exist_ok=True)

try:
    print("=" * 60)
    print("PASO 1: Mergeando LoRA con modelo base")
    print("=" * 60)
    output_path = merge_lora_with_base(LORA_PATH, str(temp_output))
    
    print("\n" + "=" * 60)
    print("PASO 2: Subiendo modelo mergeado a S3")
    print("=" * 60)
    s3_key_prefix = f"merged-models/{MODEL_NAME}"
    success = upload_to_s3(output_path, MODEL_S3_BUCKET, s3_key_prefix, AWS_REGION)
    
    if success:
        print("\n" + "=" * 60)
        print("PROCESO COMPLETADO EXITOSAMENTE")
        print("=" * 60)
        print(f"Modelo mergeado disponible en: s3://{MODEL_S3_BUCKET}/{s3_key_prefix}/")
        print(f"\nAhora puedes hacer el build con:")
        print(f"  make build-generador-image MODEL_NAME={MODEL_NAME}")
    else:
        print("\nERROR: No se pudo subir el modelo a S3")
        
finally:
    print(f"\nLimpiando archivos temporales...")
    shutil.rmtree(temp_output, ignore_errors=True)
    print("Limpieza completada")


PASO 1: Mergeando LoRA con modelo base
Mergeando LoRA desde generacion/ollama/outputs/meta-llama__Llama-3.2-3B-Instruct-6_epocas/final...
Autenticándose con Hugging Face...
Autenticación exitosa con Hugging Face
Path relativo detectado, resolviendo desde raíz del proyecto: d:\OneDrive\Documents\MAIA\Semestre 4\Despliegue de Soluciones\__REPOS__\Generacion-de-resumenes.-medicos\generacion\ollama\outputs\meta-llama__Llama-3.2-3B-Instruct-6_epocas\final
Modelo base: meta-llama/Llama-3.2-3B-Instruct
Directorio de salida: generacion\ollama\temp_merged_model
Cargando tokenizer...


d:\OneDrive\Documents\MAIA\Semestre 4\Despliegue de Soluciones\__REPOS__\Generacion-de-resumenes.-medicos\.venv_generacion\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Julian Rico\.cache\huggingface\hub\models--meta-llama--Llama-3.2-3B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Cargando configuración del modelo base...
Cargando modelo base desde HuggingFace...
NOTA: Esto puede tardar varios minutos (el modelo tiene ~5GB)...


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.61s/it]


Verificando tamaño de vocabulario...
Ajustando tamaño de embeddings: 128256 -> 128257
Tamaño de embeddings ajustado exitosamente
Verificando y limpiando adapter_config.json...
Usando configuración temporal limpia
Cargando adaptador LoRA...
Mergeando LoRA con modelo base...
NOTA: Esto puede tardar varios minutos...
Guardando modelo mergeado en generacion\ollama\temp_merged_model...
NOTA: Esto puede tardar varios minutos...
Modelo mergeado guardado exitosamente en generacion\ollama\temp_merged_model

PASO 2: Subiendo modelo mergeado a S3

Subiendo modelo a S3...
  Bucket: modelo-generador-maia-g8
  Key prefix: merged-models/meta-llama__Llama-3.2-3B-Instruct-6_epocas
  Subiendo config.json... (1/9)
  Subiendo generation_config.json... (2/9)
  Subiendo model-00001-of-00003.safetensors... (3/9)
  Subiendo model-00002-of-00003.safetensors... (4/9)
  Subiendo model-00003-of-00003.safetensors... (5/9)
  Subiendo model.safetensors.index.json... (6/9)
  Subiendo special_tokens_map.json... (7/9)
